In [7]:
import os
import glob
import numpy as np
from scipy.io import loadmat
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import timecorr as tc
import math
import scipy.spatial.distance as sd
from timecorr.helpers import isfc, autofc, mean_combine, corrmean_combine, vec2mat, reduce
corrsdir = '/Users/lucyowen/Desktop/corrs_for_j'
factors = 100

if factors == 100:
    pieman_name = '../../data/pieman_ica100.mat'
else:
    pieman_name = '../../data/pieman_data.mat'
    

pieman_data = loadmat(pieman_name)  

pieman_conds = ['word']
data = []
conds = []
for c in pieman_conds:
    if c == 'paragraph':
        if factors == 700:
            next_data = list(map(lambda i: pieman_data[c][:, i][0], np.where(np.arange(pieman_data[c].shape[1]) != 3)[0]))
        else:
            next_data = list(map(lambda i: pieman_data[c][:, i][0], np.where(np.arange(pieman_data[c].shape[1]) != 0)[0]))
    else:
        next_data = list(map(lambda i: pieman_data[c][:, i][0], np.arange(pieman_data[c].shape[1])))
    data.extend(next_data)
    conds.extend([c]*len(next_data))

all_data = np.array(data)
conds = np.array(conds)
cfun = isfc
rfun = 'eigenvector_centrality'
width = 10
wp = 'gaussian'
cond = 'intact'
level = 1

gaussian = {'name': 'Gaussian', 'weights': tc.gaussian_weights, 'params': {'var': width}}

weights_paramter = eval(wp)


weights_fun=weights_paramter['weights']
weights_params=weights_paramter['params']
combine = corrmean_combine  
group_chunks = 2


## just for word group 2
#### RUN FOR 700 FACTORS ON CLUSTER
for c in pieman_conds:
    g_inds = np.array_split(list(range(np.shape(all_data[conds == c])[0])), group_chunks)
    for i in range(2):
        data_1 = np.asarray(tc.timecorr([x for x in all_data[conds == c][g_inds[i]]], cfun=isfc, rfun=rfun,
                                                     weights_function=weights_fun, weights_params=weights_params))
        lev = np.asarray(tc.timecorr([x for x in data_1], cfun=isfc, rfun=None,
                                                     weights_function=weights_fun, weights_params=weights_params))
        corr = tc.helpers.z2r(np.mean(tc.helpers.r2z(np.stack(lev, axis=2)), axis=2))
        np.save(os.path.join(corrsdir, 'lev_2_'+c+'_'+str(i+1)+'.npy'), corr)




In [21]:
data_1.shape

(18, 300, 100)

In [19]:
import os
import glob
import numpy as np
from scipy.io import loadmat
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import timecorr as tc
import math
import scipy.spatial.distance as sd
from timecorr.helpers import isfc, autofc, mean_combine, corrmean_combine, vec2mat, reduce

corrsdir = '/dartfs/rc/lab/D/DBIC/CDL/f002s72/try_corrs'
factors = 700
if factors == 100:
    pieman_name = '../../data/pieman_ica100.mat'
else:
    pieman_name = '../../data/pieman_data.mat'



pieman_data = loadmat(pieman_name)  

pieman_conds = ['rest']
data = []
conds = []
for c in pieman_conds:
    if c == 'paragraph':
        if factors == 700:
            next_data = list(map(lambda i: pieman_data[c][:, i][0], np.where(np.arange(pieman_data[c].shape[1]) != 3)[0]))
        else:
            next_data = list(map(lambda i: pieman_data[c][:, i][0], np.where(np.arange(pieman_data[c].shape[1]) != 0)[0]))
    else:
        next_data = list(map(lambda i: pieman_data[c][:, i][0], np.arange(pieman_data[c].shape[1])))
    data.extend(next_data)
    conds.extend([c]*len(next_data))

    
all_data = np.array(data)
conds = np.array(conds)
cfun = isfc
rfun = 'eigenvector_centrality'
width = 10
wp = 'gaussian'
cond = 'intact'
level = 1

gaussian = {'name': 'Gaussian', 'weights': tc.gaussian_weights, 'params': {'var': width}}

weights_paramter = eval(wp)


weights_fun=weights_paramter['weights']
weights_params=weights_paramter['params']
combine = corrmean_combine  
group_chunks = 2


#### RUN FOR 700 FACTORS ON CLUSTER
for c in pieman_conds:
    g_inds = np.array_split(list(range(np.shape(all_data[conds == c])[0])), group_chunks)
    for i in range(2):
        if os.path.exists(os.path.join(corrsdir, 'd_1_lev_3_'+c+'_'+str(i+1)+'.npy')):
            data_1 = np.load(os.path.join(corrsdir, 'd_1_lev_3_'+c+'_'+str(i+1)+'.npy'))
        else:
            data_1 = np.asarray(tc.timecorr([x for x in all_data[conds == c][g_inds[i]]], cfun=isfc, rfun=None,
                                                         weights_function=weights_fun, weights_params=weights_params))
            np.save(os.path.join(corrsdir, 'd_1_lev_3_'+c+'_'+str(i+1)+'.npy'), data_1)
        if os.path.exists(os.path.join(corrsdir, 'd_1_r_lev_3_'+c+'_'+str(i+1)+'.npy')):
            data_1_r = np.load(os.path.join(corrsdir, 'd_1_r_lev_3_'+c+'_'+str(i+1)+'.npy'))
        else:
            data_1_r = np.asarray(reduce([x for x in data_1], rfun=rfun))
            np.save(os.path.join(corrsdir, 'd_1_r_lev_3_'+c+'_'+str(i+1)+'.npy'), data_1_r)
        if os.path.exists(os.path.join(corrsdir, 'd_2_lev_3_'+c+'_'+str(i+1)+'.npy')):
            data_2 = np.load(os.path.join(corrsdir, 'd_2_lev_3_'+c+'_'+str(i+1)+'.npy'))
        else:
            data_2 = np.asarray(tc.timecorr([x for x in data_1_r], cfun=isfc, rfun=None,
                                                         weights_function=weights_fun, weights_params=weights_params))
            np.save(os.path.join(corrsdir, 'd_2_lev_3_'+c+'_'+str(i+1)+'.npy'), data_2)
        if os.path.exists(os.path.join(corrsdir, 'd_2_r_lev_3_'+c+'_'+str(i+1)+'.npy')):
            data_2_r = np.load(os.path.join(corrsdir, 'd_2_r_lev_3_'+c+'_'+str(i+1)+'.npy'))
        else:
            data_2_r = np.asarray(reduce([x for x in data_2], rfun=rfun))
            np.save(os.path.join(corrsdir, 'd_2_r_lev_3_'+c+'_'+str(i+1)+'.npy'), data_2_r)
        lev = np.asarray(tc.timecorr([x for x in data_2_r], cfun=isfc, rfun=None,
                                                     weights_function=weights_fun, weights_params=weights_params))
        corr = tc.helpers.z2r(np.mean(tc.helpers.r2z(np.stack(lev, axis=2)), axis=2))
        np.save(os.path.join(corrsdir, 'lev_3_'+c+'_'+str(i+1)+'.npy'), corr)




IndexError: invalid index to scalar variable.

In [39]:
data_1_r.shape

AttributeError: 'list' object has no attribute 'shape'

In [40]:
import os
import glob
import numpy as np
from scipy.io import loadmat
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import timecorr as tc
import math
import scipy.spatial.distance as sd
from timecorr.helpers import isfc, autofc, mean_combine, corrmean_combine, vec2mat, reduce
corrsdir = '/Users/lucyowen/Desktop/corrs_for_j'
#corrsdir = '/dartfs-hpc/rc/home/2/f002s72/Discovery_home/try_corrs'
factors = 100
if factors == 100:
    pieman_name = '../../data/pieman_ica100.mat'
else:
    pieman_name = '../../data/pieman_data.mat'



pieman_data = loadmat(pieman_name)  

pieman_conds = ['rest']
data = []
conds = []
for c in pieman_conds:
    if c == 'paragraph':
        if factors == 700:
            next_data = list(map(lambda i: pieman_data[c][:, i][0], np.where(np.arange(pieman_data[c].shape[1]) != 3)[0]))
        else:
            next_data = list(map(lambda i: pieman_data[c][:, i][0], np.where(np.arange(pieman_data[c].shape[1]) != 0)[0]))
    else:
        next_data = list(map(lambda i: pieman_data[c][:, i][0], np.arange(pieman_data[c].shape[1])))
    data.extend(next_data)
    conds.extend([c]*len(next_data))

    
all_data = np.array(data)
conds = np.array(conds)
cfun = isfc
rfun = 'eigenvector_centrality'
width = 10
wp = 'gaussian'
cond = 'intact'
level = 1

gaussian = {'name': 'Gaussian', 'weights': tc.gaussian_weights, 'params': {'var': width}}

weights_paramter = eval(wp)


weights_fun=weights_paramter['weights']
weights_params=weights_paramter['params']
combine = corrmean_combine  
group_chunks = 2

c = 'rest'
i = 1

if os.path.exists(os.path.join(corrsdir, 'd_1_lev_3_'+c+'_'+str(i+1)+'.npy')):
    data_1 = np.load(os.path.join(corrsdir, 'd_1_lev_3_'+c+'_'+str(i+1)+'.npy'))
else:
    data_1 = np.asarray(tc.timecorr([x for x in all_data[conds == c][g_inds[i]]], cfun=isfc, rfun=None,
                                                 weights_function=weights_fun, weights_params=weights_params))
    np.save(os.path.join(corrsdir, 'd_1_lev_3_'+c+'_'+str(i+1)+'.npy'), data_1)
    

if os.path.exists(os.path.join(corrsdir, 'd_1_r_lev_3_'+c+'_'+str(i+1)+'.npy')):
    data_1_r = np.load(os.path.join(corrsdir, 'd_1_r_lev_3_'+c+'_'+str(i+1)+'.npy'))
else:
    data_1_r = np.asarray(reduce([x for x in data_1], rfun=rfun))
    np.save(os.path.join(corrsdir, 'd_1_lev_3_'+c+'_'+str(i+1)+'.npy'), data_1_r)
            
        
if os.path.exists(os.path.join(corrsdir, 'd_2_lev_3_'+c+'_'+str(i+1)+'.npy')):
    data_2 = np.load(os.path.join(corrsdir, 'd_2_lev_3_'+c+'_'+str(i+1)+'.npy'))
else:
    data_2 = np.asarray(tc.timecorr([x for x in data_1_r], cfun=isfc, rfun=None,
                                                 weights_function=weights_fun, weights_params=weights_params))
    np.save(os.path.join(corrsdir, 'd_2_lev_3_'+c+'_'+str(i+1)+'.npy'), data_2)
    

if os.path.exists(os.path.join(corrsdir, 'd_2_r_lev_3_'+c+'_'+str(i+1)+'.npy')):
    data_2_r = np.load(os.path.join(corrsdir, 'd_2_r_lev_3_'+c+'_'+str(i+1)+'.npy'))
else:
    data_2_r = np.asarray(reduce([x for x in data_2], rfun=rfun))
    np.save(os.path.join(corrsdir, 'd_2_lev_3_'+c+'_'+str(i+1)+'.npy'), data_2_r)

lev = np.asarray(tc.timecorr([x for x in data_2_r], cfun=isfc, rfun=None,
                                             weights_function=weights_fun, weights_params=weights_params))

corr = tc.helpers.z2r(np.mean(tc.helpers.r2z(np.stack(lev, axis=2)), axis=2))

np.save(os.path.join(corrsdir, 'lev_3_'+c+'_'+str(i+1)+'.npy'), corr)


In [41]:
corr

array([[ 0.24064515,  0.23470211,  0.1107802 , ...,  0.46521172,
         0.55884385,  0.37192899],
       [-0.0340098 , -0.00594736,  0.07045171, ...,  0.37214514,
         0.4468934 ,  0.23341018],
       [ 0.06192946,  0.09105865,  0.25211309, ...,  0.29723035,
         0.29228299,  0.10791072],
       ...,
       [ 0.34823788,  0.0882306 ,  0.13136054, ...,  0.0843342 ,
        -0.15418132, -0.1071372 ],
       [ 0.48898461,  0.26760594,  0.31656765, ...,  0.21862599,
         0.03337871,  0.00205412],
       [ 0.61217215,  0.46424704,  0.50432289, ...,  0.37100603,
         0.27533959,  0.21146957]])

In [33]:
for c in pieman_conds:
    g_inds = np.array_split(list(range(np.shape(all_data[conds == c])[0])), group_chunks)
    for i in range(2):
        print(i)
        print(c)

0
rest
1
rest


In [35]:
i

1

In [6]:
if os.path.exists(os.path.join(corrsdir, 'd_1_lev_3_'+c+'_'+str(i+1)+'.npy')):
    data_1 = np.load(os.path.join(corrsdir, 'd_1_lev_3_'+c+'_'+str(i+1)+'.npy'))
else:
    data_1 = np.asarray(tc.timecorr([x for x in all_data[conds == c][g_inds[i]]], cfun=isfc, rfun=rfun,
                                                 weights_function=weights_fun, weights_params=weights_params))
    np.save(os.path.join(corrsdir, 'd_1_lev_3_'+c+'_'+str(i+1)+'.npy'), data_1)

In [7]:
data_1.shape

(18, 400, 100)

In [8]:
data_1_try = np.asarray(tc.timecorr([x for x in all_data[conds == c][g_inds[i]]], cfun=isfc, rfun=None,
                                                 weights_function=weights_fun, weights_params=weights_params))

In [25]:
try_reduce = reduce([x for x in data_1_try], rfun=rfun)

In [27]:
try_reduce = np.asarray(try_reduce)

In [2]:
import os
import glob
import numpy as np
from scipy.io import loadmat
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import timecorr as tc
import math
import scipy.spatial.distance as sd
from timecorr.helpers import isfc, autofc, mean_combine, corrmean_combine, vec2mat, reduce
corrsdir = '/Users/lucyowen/Desktop/corrs_for_j'
#corrsdir = '/dartfs-hpc/rc/home/2/f002s72/Discovery_home/try_corrs'
factors = 100
if factors == 100:
    pieman_name = '../../data/pieman_ica100.mat'
else:
    pieman_name = '../../data/pieman_data.mat'



pieman_data = loadmat(pieman_name)  

pieman_conds = ['rest']
data = []
conds = []
for c in pieman_conds:
    if c == 'paragraph':
        if factors == 700:
            next_data = list(map(lambda i: pieman_data[c][:, i][0], np.where(np.arange(pieman_data[c].shape[1]) != 3)[0]))
        else:
            next_data = list(map(lambda i: pieman_data[c][:, i][0], np.where(np.arange(pieman_data[c].shape[1]) != 0)[0]))
    else:
        next_data = list(map(lambda i: pieman_data[c][:, i][0], np.arange(pieman_data[c].shape[1])))
    data.extend(next_data)
    conds.extend([c]*len(next_data))

    
all_data = np.array(data)
conds = np.array(conds)
cfun = isfc
rfun = 'eigenvector_centrality'
width = 10
wp = 'gaussian'
cond = 'intact'
level = 1

gaussian = {'name': 'Gaussian', 'weights': tc.gaussian_weights, 'params': {'var': width}}

weights_paramter = eval(wp)


weights_fun=weights_paramter['weights']
weights_params=weights_paramter['params']
combine = corrmean_combine  
group_chunks = 2


#### RUN FOR 700 FACTORS ON CLUSTER
for c in pieman_conds:
    g_inds = np.array_split(list(range(np.shape(all_data[conds == c])[0])), group_chunks)
    for i in range(2):
        
        if os.path.exists(os.path.join(corrsdir, 'd_1_lev_3_'+c+'_'+str(i+1)+'.npy')):
            data_1 = np.load(os.path.join(corrsdir, 'd_1_lev_3_'+c+'_'+str(i+1)+'.npy'))
        else:
            data_1 = np.asarray(tc.timecorr([x for x in all_data[conds == c][g_inds[i]]], cfun=isfc, rfun=rfun,
                                                         weights_function=weights_fun, weights_params=weights_params))
            np.save(os.path.join(corrsdir, 'd_1_lev_3_'+c+'_'+str(i+1)+'.npy'), data_1)
            
#         if os.path.exists(os.path.join(corrsdir, 'd_1_r_lev_3_'+c+'_'+str(i+1)+'.npy')):
#             data_1_r = np.load(os.path.join(corrsdir, 'd_1_r_lev_3_'+c+'_'+str(i+1)+'.npy'))
#         else:
#             data_1_r = np.asarray(tc.timecorr([x for x in all_data[conds == c][g_inds[i]]], cfun=isfc, rfun=rfun,
#                                                          weights_function=weights_fun, weights_params=weights_params))
#             np.save(os.path.join(corrsdir, 'd_1_lev_3_'+c+'_'+str(i+1)+'.npy'), data_1)
            
#         if os.path.exists(os.path.join(corrsdir, 'd_2_lev_3_'+c+'_'+str(i+1)+'.npy')):
#             data_2 = np.load(os.path.join(corrsdir, 'd_2_lev_3_'+c+'_'+str(i+1)+'.npy'))
#         else:  
#             data_2 = np.asarray(tc.timecorr([x for x in data_1], cfun=isfc, rfun=rfun,
#                                                      weights_function=weights_fun, weights_params=weights_params))
#             np.save(os.path.join(corrsdir, 'd_2_lev_3_'+c+'_'+str(i+1)+'.npy'))
        
#         lev = np.asarray(tc.timecorr([x for x in data_2], cfun=isfc, rfun=None,
#                                                      weights_function=weights_fun, weights_params=weights_params))
#         corr = tc.helpers.z2r(np.mean(tc.helpers.r2z(np.stack(lev, axis=2)), axis=2))
#         np.save(os.path.join(corrsdir, 'lev_3_'+c+'_'+str(i+1)+'.npy'), corr)
    



KeyboardInterrupt: 

In [ ]:
data_2 = np.load(os.path.join(os.path.join(corrsdir, 'lev_2_'+c+'_'+str(i+1)+'.npy')))

In [13]:
tc.helpers.mat2vec(data_2).shape

ValueError: operands could not be broadcast together with shapes (300,5050) (300,300) 

In [14]:
data_2 = reduce(data_2, rfun=rfun)

In [16]:
data_2.shape

(300, 100)

In [1]:

import os
import glob
import numpy as np
from scipy.io import loadmat
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import timecorr as tc
import math
import scipy.spatial.distance as sd
from timecorr.helpers import isfc, autofc, mean_combine, corrmean_combine, vec2mat, reduce
%matplotlib inline
chunk_size=6

In [2]:

def _plot_borderless_clustered(x, factor_bounds=None, factor_colors=None, savefile=None, vmin=-1, vmax=1, cbar=False, width=1000, dpi=100, cmap='Spectral_r'):
    _close_all()
    width *= (1000.0 / 775.0)  # account for border
    height = (775.0 / 755.0) * float(width) * float(x.shape[0]) / float(x.shape[1])  # correct height/width distortion

    fig = plt.figure(figsize=(width / float(dpi), height / float(dpi)), dpi=dpi)

    if len(x.shape) == 2:
        im = plt.pcolormesh(x, vmin=float(vmin), vmax=float(vmax), cmap=cmap)
    else:
        plt.imshow(x)
    ax = plt.gca()
    
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_frame_on(False)

    for _, spine in ax.spines.items():
        spine.set_visible(True)

    if factor_bounds:

        ax.hlines(factor_bounds, *ax.get_xlim(), linewidth=2, colors='w')
        ax.vlines(factor_bounds, *ax.get_ylim(), linewidth=2, colors='w')

    if factor_colors:
        for i, f in enumerate(factor_bounds[:-1]):
            rect = patches.Rectangle((f,f), factor_bounds[i+1]-f-1, factor_bounds[i+1]-f-1, linewidth=6, edgecolor=factor_colors[i],facecolor='none', zorder=2)
            ax.add_patch(rect)


    fig.set_frameon(False)
    if cbar:
        cbar = fig.colorbar(im, ax=ax)
        cbar.ax.tick_params(labelsize=30)
    
    if not savefile == None:
        fig.savefig(savefile, figsize=(width / float(dpi), height / float(dpi)), bbox_inches='tight', pad_inches=0,
                    dpi=dpi)
    return fig


In [3]:
pwd

'/Users/lucyowen/repos/timecorr-paper/code/notebooks'

In [3]:
def _close_all():
    figs = plt.get_fignums()
    for f in figs:
        plt.close(f)

In [4]:

def timepoints_corr(data, order=0):
    if not order == 0:
        for o in range(order):
            data = np.asarray(tc.timecorr([x for x in data], cfun=autofc, rfun=rfun, combine=corrmean_combine,
                                                 weights_function=weights_fun, weights_params=weights_params))
    
    return data

In [5]:
figdir = '../figs'
if not os.path.exists(figdir):
    os.mkdir(figdir)

In [18]:
#corrsdir = '/Users/lucyowen/Desktop/corrs_for_j'

corrsdir = '/dartfs-hpc/rc/home/2/f002s72/Discovery_home/try_corrs'

In [85]:
centers_name = '../../data/pieman_posterior_K700.mat'

In [86]:
centers_data = loadmat(centers_name)

In [87]:
centers = posterior['posterior']['centers'][0][0][0][0][0]
widths = np.array(list(posterior['posterior']['widths'][0][0][0][0][0][:, 0].T))

In [ ]:
nl.plotting.plot_connectome(np.eye(centers.shape[0]), centers, node_color='k', node_size=(widths ** 2) / 250)

In [89]:
widths.shape

(700, 6)

In [10]:
np.save(os.path.join(corrsdir, 'centers.npy'), centers)
np.save(os.path.join(corrsdir, 'widths.npy'), widths)

In [91]:
factors = 700

In [92]:
if factors == 100:
    pieman_name = '../../data/pieman_ica100.mat'
else:
    pieman_name = '../../data/pieman_data.mat'


In [93]:
pieman_data = loadmat(pieman_name)

In [94]:
#pieman_conds = ['intact', 'paragraph', 'word', 'rest']
pieman_conds = ['word']

In [95]:
debug = False

In [96]:
if debug:
    data = []
    conds = []
    for c in pieman_conds:
        next_data = list(map(lambda i: pieman_data[c][:, i][0][:2000,:100], np.arange(4)))
        data.extend(next_data)
        conds.extend([c]*len(next_data))
    del pieman_data

else:

data = []
conds = []
for c in pieman_conds:
    if c == 'paragraph':
        if factors == 700:
            next_data = list(map(lambda i: pieman_data[c][:, i][0], np.where(np.arange(pieman_data[c].shape[1]) != 3)[0]))
        else:
            next_data = list(map(lambda i: pieman_data[c][:, i][0], np.where(np.arange(pieman_data[c].shape[1]) != 0)[0]))
    else:
        next_data = list(map(lambda i: pieman_data[c][:, i][0], np.arange(pieman_data[c].shape[1])))
    data.extend(next_data)
    conds.extend([c]*len(next_data))
    del pieman_data

In [97]:
all_data = np.array(data)
conds = np.array(conds)


In [98]:
cfun = isfc
rfun = 'eigenvector_centrality'
width = 10
wp = 'gaussian'
cond = 'intact'
level = 1

gaussian = {'name': 'Gaussian', 'weights': tc.gaussian_weights, 'params': {'var': width}}

weights_paramter = eval(wp)


weights_fun=weights_paramter['weights']
weights_params=weights_paramter['params']
combine = corrmean_combine

## for level 1

In [99]:
group_chunks = 2

In [12]:
# #### RUN FOR 700 FACTORS ON CLUSTER
# for c in pieman_conds:
#     g_inds = np.array_split(list(range(np.shape(all_data[conds == c])[0])), group_chunks)
#     for i in range(2):
#         data = np.asarray(tc.timecorr([x for x in all_data[conds == c][g_inds[i]]], cfun=isfc, rfun=None,
#                                                  weights_function=weights_fun, weights_params=weights_params))
#         corrs = tc.helpers.z2r(np.mean(tc.helpers.r2z(np.stack(data, axis=2)), axis=2))
#         np.save(os.path.join(corrsdir, 'lev_1_'+c+'_'+str(i+1)+'.npy'), corrs)
    

## for level 2

In [100]:
#### RUN FOR 700 FACTORS ON CLUSTER
for c in pieman_conds:
    g_inds = np.array_split(list(range(np.shape(all_data[conds == c])[0])), group_chunks)
    for i in range(2):
        data_1 = np.asarray(tc.timecorr([x for x in all_data[conds == c][g_inds[i]]], cfun=isfc, rfun=rfun,
                                                     weights_function=weights_fun, weights_params=weights_params))
        lev = np.asarray(tc.timecorr([x for x in data_1], cfun=isfc, rfun=None,
                                                     weights_function=weights_fun, weights_params=weights_params))
        corr = tc.helpers.z2r(np.mean(tc.helpers.r2z(np.stack(lev, axis=2)), axis=2))
        np.save(os.path.join(corrsdir, 'lev_2_'+c+'_'+str(i+1)+'.npy'), corr)
    

In [ ]:
## just for word group 2
#### RUN FOR 700 FACTORS ON CLUSTER
for c in pieman_conds:
    g_inds = np.array_split(list(range(np.shape(all_data[conds == c])[0])), group_chunks)
    i = 1 
    data_1 = np.asarray(tc.timecorr([x for x in all_data[conds == c][g_inds[i]]], cfun=isfc, rfun=rfun,
                                                 weights_function=weights_fun, weights_params=weights_params))
    lev = np.asarray(tc.timecorr([x for x in data_1], cfun=isfc, rfun=None,
                                                 weights_function=weights_fun, weights_params=weights_params))
    corr = tc.helpers.z2r(np.mean(tc.helpers.r2z(np.stack(lev, axis=2)), axis=2))
    np.save(os.path.join(corrsdir, 'lev_2_'+c+'_'+str(i+1)+'.npy'), corr)
    

## for level 3

In [13]:
#### RUN FOR 700 FACTORS ON CLUSTER
for c in pieman_conds:
    g_inds = np.array_split(list(range(np.shape(all_data[conds == c])[0])), group_chunks)
    for i in range(2):
        data_1 = np.asarray(tc.timecorr([x for x in all_data[conds == c][g_inds[i]]], cfun=isfc, rfun=rfun,
                                                     weights_function=weights_fun, weights_params=weights_params))
        data_2 = np.asarray(tc.timecorr([x for x in data_1], cfun=isfc, rfun=rfun,
                                                     weights_function=weights_fun, weights_params=weights_params))
        lev = np.asarray(tc.timecorr([x for x in data_2], cfun=isfc, rfun=None,
                                                     weights_function=weights_fun, weights_params=weights_params))
        corr = tc.helpers.z2r(np.mean(tc.helpers.r2z(np.stack(lev, axis=2)), axis=2))
        np.save(os.path.join(corrsdir, 'lev_3_'+c+'_'+str(i+1)+'.npy'), corr)
    

In [42]:
corr_breakdown = corr

data_1 = np.asarray(tc.timecorr([x for x in all_data[conds == c][g_inds[i]]], cfun=isfc, rfun=rfun,
                                             weights_function=weights_fun, weights_params=weights_params))
data_2 = np.asarray(tc.timecorr([x for x in data_1], cfun=isfc, rfun=rfun,
                                             weights_function=weights_fun, weights_params=weights_params))
lev = np.asarray(tc.timecorr([x for x in data_2], cfun=isfc, rfun=None,
                                             weights_function=weights_fun, weights_params=weights_params))
corr = tc.helpers.z2r(np.mean(tc.helpers.r2z(np.stack(lev, axis=2)), axis=2))
np.save(os.path.join(corrsdir, 'lev_3_'+c+'_'+str(i+1)+'.npy'), corr)


## For sliced correlations:

In [81]:
percentages = np.arange(.1, 1.1, .1)
for c in pieman_conds:
    for g in glob.glob(os.path.join(corrsdir, 'lev_2_'+c+'*.npy')):
        corrs = np.load(g)
        mat_corrs = tc.helpers.vec2mat(corrs)
        
        sliced_corrs = np.array([])
        
        for p in percentages:
            s = int(p*np.shape(mat_corrs)[-1])
            slice_corr = mat_corrs[:, :, s-1]
            if sliced_corrs.shape[0] == 0:
                sliced_corrs = slice_corr
            else:
                sliced_corrs = np.dstack((sliced_corrs, slice_corr))
            np.save(os.path.splitext(g)[0] + '_tenths.npy', sliced_corrs)


In [83]:
sliced_corrs.shape


(700, 700, 10)

## For averaged correlations:

In [ ]:
percentages = np.arange(.1, 1.1, .1)
for c in pieman_conds:
    for g in glob.glob(os.path.join(corrsdir, 'lev_2_'+c+'*.npy')):
        corrs = np.load(g)
        mat_corrs = tc.helpers.vec2mat(corrs)
        
        sliced_corrs = np.array([])
        
        for p in percentages:
            s = int(p*np.shape(mat_corrs)[-1])
            slice_corr = mat_corrs[:, :, s-1]
            if sliced_corrs.shape[0] == 0:
                sliced_corrs = slice_corr
            else:
                sliced_corrs = np.dstack((sliced_corrs, slice_corr))
            np.save(os.path.splitext(g)[0] + '_tenths.npy', sliced_corrs)